In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras 
from keras.layers import Input, Dense, Dropout, LSTM
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model

In [2]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])

    return np.array(data), np.array(labels)

In [3]:
df = pd.read_csv("../data/data_processed_cleaned.csv")
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d %H:%M')

df['fve'] = df['fve'] * 0.03
df['mve'] = df['mve'] * 0.07
df['wpg'] = df['wpg'] * 0.07

df['production_usage'] = df['fve'] + df['mve'] + df['wpg'] 
TRAIN_SPLIT = 29000

In [4]:
features_considered = ['price', 'isWeekend', 'isHoliday','temp', 'wind', 'day_cos', 'day_sin', 'month_cos', 'month_sin']
features = df[features_considered]
features.index = df['date']

prod_dataset = df['production_usage'].values
data_mean = prod_dataset.mean(axis=0)
data_std = prod_dataset.std(axis=0)
prod_dataset = (prod_dataset-data_mean)/data_std


cons_dataset = df['consumption_usage'].values
data_mean = cons_dataset.mean(axis=0)
data_std = cons_dataset.std(axis=0)
cons_dataset = (cons_dataset-data_mean)/data_std

dataset = features.values
data_mean = dataset.mean(axis=0)
data_std = dataset.std(axis=0)
dataset = (dataset-data_mean)/data_std

In [5]:
past_history = 96
future_target = 4
STEP = 1


x, y1 = multivariate_data(dataset, prod_dataset, 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
x, y2 = multivariate_data(dataset, cons_dataset, 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
x_val , y1_val = multivariate_data(dataset, prod_dataset,
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)
x_val , y2_val = multivariate_data(dataset, cons_dataset,
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

In [9]:
# https://stackoverflow.com/questions/47596703/multiple-outputs-for-multi-step-ahead-time-series-prediction-with-keras-lstm

inputs = Input(shape=(past_history,len(features_considered))) 
m = LSTM(64,  return_sequences=True)(inputs)
m = Dropout(0.3)(m)
m = LSTM(64)(m)
m = Dropout(0.2)(m)
outputA = Dense(future_target, activation='tanh')(m)
outputB = Dense(future_target, activation='tanh')(m)

m = Model(inputs=[inputs], outputs=[outputA, outputB])
m.compile(optimizer='adam', loss='mse')


history = m.fit(x,[y1,y2], epochs=10,steps_per_epoch=100)

test_scores = m.evaluate(x_val, [y1_val, y2_val], verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/10
100/100 [==============================] - 8s 68ms/step - loss: 1.1658 - dense_5_loss: 0.6643 - dense_6_loss: 0.5015
Epoch 2/10
100/100 [==============================] - 6s 62ms/step - loss: 0.7698 - dense_5_loss: 0.4197 - dense_6_loss: 0.3500
Epoch 3/10
100/100 [==============================] - 6s 62ms/step - loss: 0.6701 - dense_5_loss: 0.3473 - dense_6_loss: 0.3228
Epoch 4/10
100/100 [==============================] - 6s 62ms/step - loss: 0.5849 - dense_5_loss: 0.2915 - dense_6_loss: 0.2934
Epoch 5/10
100/100 [==============================] - 6s 62ms/step - loss: 0.5329 - dense_5_loss: 0.2741 - dense_6_loss: 0.2589
Epoch 6/10
100/100 [==============================] - 6s 62ms/step - loss: 0.5033 - dense_5_loss: 0.2612 - dense_6_loss: 0.2422
Epoch 7/10
100/100 [==============================] - 6s 62ms/step - loss: 0.4669 - dense_5_loss: 0.2475 - dense_6_loss: 0.2194
Epoch 8/10
100/100 [==============================] - 6s 62ms/step - loss: 0.4440 - dense_5_loss: 0.2365